In [ ]:
# Download Libraries
!pip3 install google-cloud-storage
!pip3 install pyarrow # Apache Arrow

: 

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import io
from io import StringIO
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
import os
import datetime
from google.cloud import bigquery
import unicodedata

In [ ]:
#Gathering Data
#Extracting from datasource to view the head 

URL = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Define the data type for column 17 because it contains mixed data types including NaN
dtype_mapping = {'Veteran Credit': 'object'}

# Specify NaN values to be considered as missing values
na_values = ['NaN', '', 'NA', 'nan']

df_raw = pd.read_csv(URL, dtype=dtype_mapping, na_values=na_values, low_memory=False)

print(df_raw.info())
print('\n')
print(df_raw.shape)
print('\n')
df_raw.head()

In [ ]:
#Storing Data
# Specify the path to your service account key file
service_account_key_file = '/Users/karmayangchentenzin/Downloads/service_acc_key.json'

# Optionally, load other configuration settings from the JSON file
with open(service_account_key_file, 'r') as f:
    config = json.load(f)

# Fetch data from the web (CSV file)
url = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Read the contents of the CSV file
response = requests.get(url)

# Check if the request was successful
if response.status_code != 200:
    print("Failed to fetch CSV file.")
    exit()

# Read the CSV data into a DataFrame
try: 
    df = pd.read_csv(io.StringIO(response.text), dtype=dtype_mapping, na_values=na_values, low_memory=False)
    # Add timestamp column
    df['load_date'] = datetime.datetime.now() #Date the data is stored. A new column named "load_data" is created.

    # Replace periods with underscores in column names #This had to be done before the original data was loaded from the source to GC because when creating tables in BigQuery, an error message was shown saying that field names can not contain a period (.)
    df.columns = df.columns.str.replace('.', '_')

    # Convert DataFrame to PyArrow Table
    table = pa.Table.from_pandas(df)

    # Write PyArrow Table to Parquet format
    parquet_file_name = 'ny_civil_service_exam.parquet'  # name for the Parquet file
    pq.write_table(table, parquet_file_name)

    # Upload Parquet file to Google Cloud Storage
    bucket_name = 'cis4400_hw1_kyt'
    blob_name = parquet_file_name

    client = storage.Client.from_service_account_json(service_account_key_file)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(parquet_file_name)

    print(f"Parquet file '{parquet_file_name}' uploaded to {bucket_name}/{blob_name} in Google Cloud Storage.")
except Exception as e:
    print(f"Error occurred: {e}")


In [ ]:
# Display DataFrame with added timestamp column
print(df.head())

# Verify Parquet file contents
parquet_table = pq.read_table(parquet_file_name)
parquet_df = parquet_table.to_pandas()
print(parquet_df.head())